In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Reference for EDA:
https://www.kaggle.com/ambrosm/tpsjan22-01-eda-which-makes-sense


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import seaborn as sns
import re
import datetime
# from sklearn.model_selection import train_test_split

In [ ]:
train_data=pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv")

In [ ]:
train_data.head(), train_data.tail()

In [ ]:
countries=set(train_data['country'])
countries

In [ ]:
products=set(train_data['product'])
products

In [ ]:
store_type=set(train_data['store'])
store_type

<h2> Gist of the the Data </h2>
<h4>
Row Ids <br>
Datetime Column indicating this is a Time Series Data <br>
Countries - Finland, Norway, Sweeden <br>
Store Types - Kaggle Mart and Kaggle Rama <br>
Products - Hats, Mugs, Stickers <br>
</h4>


<h2> EDA </h2>

In [ ]:
# Country -wise Sale 
train_data['country'].value_counts()
# Inference -> Same sales  in each of the Country

In [ ]:
# Product Wise Sale
train_data['product'].value_counts()
# Inference -> Same sales of each of the three Product

In [ ]:
# Store wise Sale
train_data['store'].value_counts()
# Inference -> Same sales at both the types of stores

In [ ]:
print('Train data dates ranging from:', train_data['date'].min(), '->', train_data['date'].max())

In [ ]:
train_data['date'] = pd.to_datetime(train_data['date'])

In [ ]:
train_data['Year'] = train_data.date.dt.year
train_data['Month'] = train_data.date.dt.month

Since this is a Time Series Forecasting Question the Test Dates would be dates in the 
future from 2019.

In [ ]:
# First let us Plot the Frequencies of Sales throughout the Time Frame
plt.figure(figsize=(9,9))
sns.lineplot(x = 'date', y = 'num_sold', data = train_data, label = 'daily')
# Inference -> December - January Festive season records the Highest amount of Sales
# April Easter Season gets second hgihest Spike in Sales

In [ ]:
plt.figure(figsize=(9,9))
sns.lineplot(x = 'date', y = 'num_sold', data = train_data,hue='store')
# Inference -> Kaggle RAMA Sales > Kaggle Mart Sales

In [ ]:
plt.figure(figsize=(9,9))
sns.lineplot(x = 'date', y = 'num_sold', data = train_data,hue='product')
# Inference ->  Sales of Hats > Sales of Mugs > Sales of Stickers

In [ ]:
plt.figure(figsize=(9,9))
sns.lineplot(x = 'date', y = 'num_sold', data = train_data,hue='country')
# Inference ->  Sales of Norway > Sales of Sweden > Sales of Finland

In [ ]:
plt.figure(figsize=(9,9))
sns.lineplot(x = 'Year', y = 'num_sold', data = train_data,hue='country')
# sns.lineplot(x = 'month', y = 'num_sold', data = train_data,hue='country')
# Inference ->  Sales of Norway > Sales of Sweden > Sales of Finland

In [ ]:
train_data.describe()
# Neglect it for row_id
# Daily Average for items Sold-> 388

In [ ]:
# Get a General trend of Increase in Sales, so the test data will definitely be an extrapolation
plt.figure(figsize=(9,9))
sns.lineplot(x = 'Year', y = 'num_sold', data = train_data,hue='store')

In [ ]:
#  Now we will Segregate the Data Country-wise and further do some Visualization before moving
# to Feature Engineering
train_data_Finland = train_data[train_data.country == "Finland"]
train_data_Norway = train_data[train_data.country == "Norway"]
train_data_Sweden = train_data[train_data.country == "Sweden"]

In [ ]:
# Now we will analyse the trends for each Country as Well
train_data_Finland['num_sold'].describe(),train_data_Norway['num_sold'].describe(),train_data_Sweden['num_sold'].describe()

In [ ]:
train_monthly_country = train_data.set_index('date').groupby([pd.Grouper(freq = 'M'), 'country'])[['num_sold']].mean()
plt.figure(figsize = (9, 9))
sns.lineplot(x = 'date', y = 'num_sold', hue = 'country', data = train_monthly_country)

In [ ]:
train_data.groupby(['country', 'store', 'product']).num_sold.agg(['max', 'mean'])

<h2> Data Cleaning and Feature Engineering

<h4>
So obviously there ia a Trend of Norway > Sweden > Finland <br>
So the Feature Engineering further will require a row of GDP Like how a lot of people have done <br>
So we will have to bring the GDP Dataset as an added feature for the Model <br>


In [ ]:
# Importing the Datasets for completeness
train=pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test=pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
gdp=pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')
sample_sub=pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

In [ ]:
train.head(),test.head(),gdp.head(),sample_sub.head()

In [ ]:
# Defining sMAPE
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)
def SMAPE2(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return diff.squeeze()


In [ ]:
import holidays
import lightgbm
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
import dateutil.easter as easter

In [ ]:
# Feature engineering


In [ ]:
#  Last Cell
# sub['num_sold'] = np.round(sub['num_sold']).astype(int)
# import pandas as pd
# sub = pd.read_csv('../input/tps-jan-2022-automated-ensembling/submission.csv')
# sub.num_sold *= 0.99
# sub.to_csv('submission.csv', index=False)
# sub